In [3]:
!pip install tqdm
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import stats

from IPython.display import display
from IPython import get_ipython
get_ipython().run_line_magic('matplotlib', 'inline')

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from tqdm import tqdm 
from time import time

from sklearn.metrics import accuracy_score, f1_score

from sklearn.ensemble import GradientBoostingRegressor
from matplotlib import pyplot

You are using pip version 9.0.1, however version 9.0.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [15]:
housing_df = pd.read_pickle('Data/housing_df.pkl')
def apply_scale(dataframe, scaling_function):
    numerical_df = dataframe.select_dtypes(include=[float])
    print(numerical_df.columns)
    numerical_df = scaling_function(numerical_df)
    tmp_df = dataframe.copy()
    tmp_df[numerical_df.columns] = numerical_df
    return tmp_df

def gelman_scale(dataframe):
    return (dataframe - dataframe.mean())/(2*dataframe.std())

def standard_scale(dataframe):
    return (dataframe - dataframe.mean())/(dataframe.std())

df_gelman = apply_scale(housing_df, gelman_scale)
housing_train_df = df_gelman.drop('SalePrice', axis=1)
housing_target = df_gelman['SalePrice']

Index(['Mas Vnr Area', 'Bsmt Unf SF', 'Total Bsmt SF', 'Garage Area'], dtype='object')


In [16]:
def mean_r2_for_feature(data, feature):
    scores = []
    for _ in range(10):
        scores.append(calculate_r_2_for_feature(data, feature))
        
    scores = np.array(scores)
    return scores.mean()

def calculate_r_2_for_feature(data,feature):
    new_data = data.drop(feature, axis=1)

    X_train, \
    X_test,  \
    y_train, \
    y_test = train_test_split(
        new_data,data[feature],test_size=0.33
    )

    regressor = GradientBoostingRegressor()
    regressor.fit(X_train,y_train)
    
    score = regressor.score(X_test,y_test)
    return score

r2_results = {}
numeric_cols = housing_df.drop(['SalePrice'], axis=1).select_dtypes(exclude=['object']).columns
print("Calculate R2 Score for {} features".format(len(numeric_cols)))
for col in numeric_cols:
    r2 = mean_r2_for_feature(housing_train_df, col)
    print("{:24} {}".format(col, r2))
    r2_results[col] = r2

Calculate R2 Score for 1922 features
MS SubClass              0.962221983353394
Lot Area                 0.37042712207050826
Overall Qual             0.6664083158340643
Overall Cond             0.35764364260493214
Mas Vnr Area             0.5404577340247699
Bsmt Unf SF              0.8096989312034653
Total Bsmt SF            0.8313393617212619
1st Flr SF               0.9288335219197409
2nd Flr SF               0.9736709576485545
Low Qual Fin SF          0.16323443674984217
Gr Liv Area              0.9694187092474584
Garage Area              0.800674637644026
Wood Deck SF             0.09199488956379279
Open Porch SF            0.13005961289794504
Enclosed Porch           0.03331852499153117
3Ssn Porch               -0.37497584606536377
Screen Porch             -0.019783149110530585
Pool Area                0.9216933582627064
Misc Val                 0.27740303497738017
MS Zoning_C (all)        0.7831673134408185
MS Zoning_FV             0.9999979197562572
MS Zoning_I (all)        -0.0

KeyboardInterrupt: 

In [ ]:
corr = df_gelman.corr()
corr['SalePrice-Abs'] = abs(corr['SalePrice'])
corr.sort_values(by=['SalePrice-Abs'], ascending=False)['SalePrice-Abs'][0:20]